In [188]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [189]:
def get_players():

    url = "https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats"

    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    players = soup.find_all("td", {"class":"left", "data-stat":"player"})
    positions = soup.find_all("td", {"class":"center", "data-stat":"position"})#
    teams = soup.find_all("td", {"class":"left", "data-stat":"team"})

    players_list = []
    for i in range(len(players)):
        name = players[i].find("a").text
        position = positions[i].text
        team = teams[i].text
        href = players[i].find("a").get("href")
        players_list.append([name, position, team, href])
    df = pd.DataFrame(players_list, columns=["Name", "Position", "Team", "URL"])
    #df["Name"] = df["Name"].str.replace(" ", "-")
    df["Position"] = df["Position"].str.split(",").apply(lambda x: x[0])
    
    df.set_index("Name", inplace=True)
    
    return df

In [194]:
def player_stats(player_name):
    
    player_database = get_players()
    
    url = player_database.loc[player_name]["URL"]
    
    # Define the URL of the page you want to scrape
    url = "https://fbref.com/{}".format(url)

    # Send a GET request to the website
    response = requests.get(url)

    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table containing the performance statistics
    table = soup.find('table', {'id': 'scout_summary_MF'})

    # Find all the rows in the table
    rows = table.find_all('tr')

    import pandas as pd

    # Extract the data from the rows of the table
    data = []
    index = []
    for row in rows:
        index.append(row.find('th').text)
        data.append([cell.text for cell in row.find_all('td')])

    # Extract the column titles from the table
    column_titles = [cell.text for cell in table.find('thead').find_all('th')]

    # create index dataframe
    index_df = pd.DataFrame(index, columns=[column_titles[0]])

    # Create a dataframe from the extracted data
    data_df = pd.DataFrame(data, columns=column_titles[1:])

    # concatenate index_df and data_df
    stats = pd.concat([index_df, data_df], axis=1)
    stats = stats.drop(stats.index[0]).set_index("Statistic", drop=True)
    stats = stats[stats.astype(bool)].dropna()
    stats["Per 90"]["Pass Completion %"] = stats["Per 90"]["Pass Completion %"].replace("%", "")
    stats = result.astype("float")
    
    return stats

In [195]:
player_stats("Kalvin Phillips")

,Per 90,Percentile
Statistic,,
Non-Penalty Goals,0.00,12.0
Non-Penalty xG,0.03,9.0
Shots Total,0.44,7.0
Assists,0.00,11.0
xAG,0.01,2.0
npxG + xAG,0.03,1.0
Shot-Creating Actions,1.31,8.0
Passes Attempted,51.41,55.0
Pass Completion %,79.00,33.0
